In [3]:
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import h5py
from random import shuffle
import glob
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
file_handle = h5py.File('D:/GOLD_XYZ_OSC.0001_1024.hdf5', 'r')

myData = file_handle['X'][:]  #1024x2 samples
myMods = file_handle['Y'][:]  #mods
mySNRs = file_handle['Z'][:]  #snrs

print(np.shape(myData))
print(np.shape(myMods))
print(np.shape(mySNRs))
file_handle.close()

np.random.seed(0)

(2555904, 1024, 2)
(2555904, 24)
(2555904, 1)


In [4]:
snrs = list(np.unique(mySNRs.T[0]))
print(snrs)

[-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]


In [5]:
mods = ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
                           '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
                           '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC',
                           'FM', 'GMSK', 'OQPSK']
num_classes=np.shape(mods)[0]
print("the number of classes is",num_classes)

the number of classes is 24


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 确保 myMods 是一维类别标签，而不是 one-hot 编码
myMods = np.argmax(myMods, axis=1) if myMods.ndim > 1 else myMods.flatten()
mySNRs = mySNRs.flatten()

# 创建一个 DataFrame，便于操作和分组采样
data_df = pd.DataFrame({
    'mod': myMods,
    'snr': mySNRs,
    'index': np.arange(len(myMods))  # 添加索引用于回溯
})

# 按调制方式和 SNR 分组，每组随机采样 10%
grouped = data_df.groupby(['mod', 'snr'])
sampled_indices = grouped.sample(frac=0.045, random_state=42)['index']  # 采样 10%

# 从采样后的数据中划分训练集和测试集
sampled_data = data_df.loc[sampled_indices]
train_indices, test_indices = train_test_split(
    sampled_data['index'].values, test_size=0.3, random_state=42, stratify=sampled_data[['mod', 'snr']]
)

# 根据索引提取训练集和测试集
X_train = myData[train_indices]
Y_train = myMods[train_indices]
Z_train = mySNRs[train_indices]

X_test = myData[test_indices]
Y_test = myMods[test_indices]
Z_test = mySNRs[test_indices]

# 打印结果形状
print(f"Training Data Shape: {X_train.shape}")
print(f"Training Labels Shape: {Y_train.shape}")
print(f"Training SNR Shape: {Z_train.shape}")
print(f"Testing Data Shape: {X_test.shape}")
print(f"Testing Labels Shape: {Y_test.shape}")
print(f"Testing SNR Shape: {Z_test.shape}")



Training Data Shape: (80371, 1024, 2)
Training Labels Shape: (80371,)
Training SNR Shape: (80371,)
Testing Data Shape: (34445, 1024, 2)
Testing Labels Shape: (34445,)
Testing SNR Shape: (34445,)


In [7]:
X_train = X_train.reshape(X_train.shape[0], 32, 32, 2)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 2)

# 打印结果形状
print(f"Training Data Shape: {X_train.shape}")
print(f"Training Labels Shape: {Y_train.shape}")
print(f"Training SNR Shape: {Z_train.shape}")
print(f"Testing Data Shape: {X_test.shape}")
print(f"Testing Labels Shape: {Y_test.shape}")
print(f"Testing SNR Shape: {Z_test.shape}")

Training Data Shape: (80371, 32, 32, 2)
Training Labels Shape: (80371,)
Training SNR Shape: (80371,)
Testing Data Shape: (34445, 32, 32, 2)
Testing Labels Shape: (34445,)
Testing SNR Shape: (34445,)


In [8]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 699.0 kB/s eta 0:02:58
   ---------------------------------------- 0.5/124.9 MB 699.0 kB/s eta 0:02:58
   ---------------------------------------- 0.5/124.9 MB 699.0 kB/s eta 0:02:58
   ---------------------------------------- 0.8/124.9 MB 459.9 kB/s eta 0:04:30
   ---------------------------------------- 0.8/124.9 MB 459.9 kB/s eta 0:04:30
   ---------------------------------------- 0.8/124.9 MB 459.9 kB/s eta 0:04:30
   ---------------------------------------- 0.8/124.9 MB 459.9 kB/s eta 0:04:30
   ---------------------------------------- 1.0/124.9 MB 406.0 kB/s eta 0:05:06
   ---------------------------------------- 1.0/124.9 MB 406.0 kB/s eta 0:05

In [11]:
# Alex
# yifanwang0916@outlook.com
# 2019.09.25

import numpy as np
import cv2
import time

from framework.layer import *
from framework.utli import *
from framework.pixelhop import *
# from framework.data import *
from framework.LAG import LAG_Unit

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from skimage.measure import block_reduce
class_list = list(range(24))
def myModel(x, getK=1):
    x1 = PixelHop_Unit(x, dilate=1, pad='reflect', num_AC_kernels=9, weight_name='pixelhop1.pkl', getK=getK)

    x2 = PixelHop_Unit(x1, dilate=2, pad='reflect', num_AC_kernels=15, weight_name='pixelhop2.pkl', getK=getK)
    x2 = AvgPooling(x2)

    x3 = PixelHop_Unit(x2, dilate=2, pad='reflect', num_AC_kernels=25, weight_name='pixelhop3.pkl', getK=getK)
    x3 = AvgPooling(x3)

    x4 = PixelHop_Unit(x3, dilate=2, pad='reflect', num_AC_kernels=30, weight_name='pixelhop4.pkl', getK=getK)

    x2 = myResize(x2, x.shape[1], x.shape[2])
    x3 = myResize(x3, x.shape[1], x.shape[2])
    x4 = myResize(x4, x.shape[1], x.shape[2])
    return np.concatenate((x1,x2,x3,x4), axis=3)


SAVE={}
train_feature1 = PixelHop_Unit(X_train, dilate=1, pad='reflect', num_AC_kernels=9, weight_name='pixelhop1_mnist.pkl', getK=1)
train_feature = block_reduce(train_feature1, (1, 4, 4, 1), np.mean).reshape(len(X_train),-1)
train_feature_reduce=LAG_Unit(train_feature,train_labels=Y_train, class_list=class_list,
                          SAVE=SAVE,num_clusters=160,alpha=5,Train=True)

test_feature1 = PixelHop_Unit(X_test, dilate=1, pad='reflect', num_AC_kernels=9, weight_name='pixelhop1_mnist.pkl', getK=0)
test_feature=block_reduce(test_feature1, (1, 4, 4, 1), np.mean).reshape(len(X_test),-1)
test_feature_reduce=LAG_Unit(test_feature,train_labels=None, class_list=class_list,
                      SAVE=SAVE,num_clusters=160,alpha=5,Train=False)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
scaler=preprocessing.StandardScaler()
feature = scaler.fit_transform(train_feature_reduce)
feature_test = scaler.transform(test_feature_reduce)

import xgboost as xgb
clf = xgb.XGBClassifier(n_jobs=-1,
                    objective='binary:logistic',
                    # tree_method='gpu_hist', gpu_id=None,
                    max_depth=20,n_estimators=200,
                    min_child_weight=5,gamma=8,
                    subsample=0.8,learning_rate=0.008,
                    nthread=8,colsample_bytree=1.0)
clf.fit(feature, Y_train)
##        clf=RandomForestClassifier(n_estimators=500,max_depth=5).fit(train_f, train_labels)
print('***** Train ACC:', accuracy_score(Y_train,clf.predict(feature)))
print('***** Test ACC:', accuracy_score(Y_test,clf.predict(feature_test)))

=========== Start: PixelHop_Unit
------------------- Start: PixelHop_8_Neighbour
       <Info>        Input feature shape: (80371, 32, 32, 2)
       <Info>        dilate: 1
       <Info>        padding: reflect
       <Info>        Output feature shape: (80371, 32, 32, 18)
------------------- End: PixelHop_8_Neighbour -> using  45.084090 seconds
------------------- Start: Saab transformation
       <Info>        pixelhop_feature.shape: (80371, 32, 32, 18)


MemoryError: Unable to allocate 11.0 GiB for an array with shape (80371, 32, 32, 18) and data type float64